In [5]:
!ls

0. Main program.txt
0. Source code_Walkthrough_Delivery1&2.ipynb
0. Source code_Walkthrough_Delivery3.ipynb
1. Data_Preparation_V1a_to debug.ipynb
1. Data_Preparation_V1b_to debug.ipynb
1.A Data_Preparation.py
1.B Data_Preparation.py
2.A Data Understanding_EDA.py
2.A Data_Understanding_EDA_1.0.ipynb
2.A.1 Dual dashboard.py
SIR_modeling_2022_exercise.ipynb
reference help


In [6]:
import pandas as pd
import numpy as np

from datetime import datetime

import matplotlib as mpl
import matplotlib.pyplot as plt

from datetime import datetime

%matplotlib inline
pd.set_option('display.max_rows', 500)

![CRISP_DM](../reports/figures/CRISP_DM.png)

# Data Preparation

* focus is always to understand the final data structure
* support each step by visual analytics

## Data reference
### Johns Hopkins GITHUB csv data

https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series
|-> Infections

### Centers for Civic Impact

https://github.com/govex/COVID-19/blob/a96dbc70eada30e83b1c475a328bb3cab4712741/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv |-> Vaccinned

## 1 Get data and prepare delivery 3 

In [7]:
# %load /Users/victhorvic/ads_covid-19/src/data/get_data_all_countries.py
#!/usr/bin/env python3
"""
Created on Mon Jul  4 18:59:23 2022

@author: victhorvic
"""
# %% [0] Libraries
import pandas as pd
import numpy as np

from datetime import datetime


import matplotlib.pyplot as plt

#%% [1] Get data

'''
Links of the data

Johns Hopkins GITHUB csv data
https://github.com/CSSEGISandData/COVID-19/tree/master/csse_covid_19_data/csse_covid_19_time_series |-> Infections

Centers for Civic Impact
https://github.com/govex/COVID-19/blob/a96dbc70eada30e83b1c475a328bb3cab4712741/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv |-> Vaccinned
'''

data_path = 'https://github.com/CSSEGISandData/COVID-19/blob/246eab67395dce9a4238fff77aa5f3561e253d48/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv?raw=true'
## added ?raw=true to get last value and get the permantlink from Github

#data_path1 = 'https://github.com/govex/COVID-19/blob/a96dbc70eada30e83b1c475a328bb3cab4712741/data_tables/vaccine_data/global_data/time_series_covid19_vaccine_global.csv?raw=true'
## added ?raw=true to get last value and get the permantlink from Github


# data infections 
pd_raw_inf=pd.read_csv(data_path,)


# %% [3.1] Infections data
'''Data for infections COVID¶'''

# define time index
time_idx=pd_raw_inf.columns[4:]

df_plot= pd.DataFrame({'date':time_idx})

# define countries
country_list=['Mexico',
              'US',
              'Germany',
             ] 
country_list = pd_raw_inf['Country/Region'].unique()

for each in country_list:
    df_plot[each] = np.array(pd_raw_inf[pd_raw_inf['Country/Region']==each].iloc[:,4::].sum(axis=0))


# %%% [3.3] Save to csv
'''Export csv file'''


''' Fix time_index'''
time_idx=[datetime.strptime( each,"%m/%d/%y") for each in df_plot.date] 
# convert to datetime

time_str=[each.strftime('%Y-%m-%d') for each in time_idx] 
# convert back to date ISO norm (str)


df_plot['date']=time_idx
type(df_plot['date'][0])


df_plot.to_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_inf_small_flat_table2.csv',sep=';',index=False)

# %% [4] Create relational model

''' Transformes the COVID data in a relational data set'''

#data_path='data/raw/COVID-19/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv'
pd_raw=pd.read_csv(data_path)

pd_data_base=pd_raw.rename(columns={'Country/Region':'country',
                      'Province/State':'state'})

pd_data_base['state']=pd_data_base['state'].fillna('no')

pd_data_base=pd_data_base.drop(['Lat','Long'],axis=1)


pd_relational_model=pd_data_base.set_index(['state','country']) \
                                .T                              \
                                .stack(level=[0,1])             \
                                .reset_index()                  \
                                .rename(columns={'level_0':'date',
                                                   0:'confirmed'},
                                                  )

pd_relational_model['date']=pd_relational_model.date.astype('datetime64[ns]')

pd_relational_model.to_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_relational_confirmed.csv',sep=';',index=False)
#print(' Number of rows stored: '+str(pd_relational_model.shape[0]))
#print(' Latest date is: '+str(max(pd_relational_model.date)))


/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_plot[each] = np.array(pd_raw_inf[pd_raw_inf['Country/Region']==each].iloc[:,4::].sum(axis=0))
/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_plot[each] = np.array(pd_raw_inf[pd_raw_inf['Country/Region']==each].iloc[:,4::].sum(axis=0))
/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: Pe

/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_plot[each] = np.array(pd_raw_inf[pd_raw_inf['Country/Region']==each].iloc[:,4::].sum(axis=0))
/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_plot[each] = np.array(pd_raw_inf[pd_raw_inf['Country/Region']==each].iloc[:,4::].sum(axis=0))
/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: Pe

/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_plot[each] = np.array(pd_raw_inf[pd_raw_inf['Country/Region']==each].iloc[:,4::].sum(axis=0))
/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  df_plot[each] = np.array(pd_raw_inf[pd_raw_inf['Country/Region']==each].iloc[:,4::].sum(axis=0))
/var/folders/z_/gl6xz9_x40x19f9gt0t5bjph0000gn/T/ipykernel_4243/2117395445.py:56: Pe

## 2 Build Features

Prepare data filtered and double rated

In [ ]:
# %load /Users/victhorvic/ads_covid-19/src/features/build_features_2.py
#!/usr/bin/env python3
"""
Created on Mon Jul  4 18:59:23 2022

@author: victhorvic

This program is to gete data filtered and double rated 2nd way

"""
# %% [0] Libraries
import pandas as pd
import numpy as np
from sklearn import linear_model
reg = linear_model.LinearRegression(fit_intercept=True)

from scipy import signal

import matplotlib.pyplot as plt
import plotly.graph_objects as go

# %% [1] Get data

# try to parse the dates right at the beginning 
# it works out of the box if the date was stored ISO YYYY-MM-DD format

# infections
#df_analyse = pd.read_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_inf_small_flat_table2.csv',
#                           sep=';', parse_dates=[0])

#Data from get_data_all_countries.py
df_analyse = pd.read_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_relational_confirmed.csv',sep=';', parse_dates = [0])

df_analyse2 = pd.read_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_inf_small_flat_table2.csv',sep=';', )

# %% [2] Functions 

def get_doubling_time_via_regression(in_array):
    ''' Use a linear regression to approximate the doubling rate

        Parameters:
        ----------
        in_array : pandas.series

        Returns:
        ----------
        Doubling rate: double
    '''

    y = np.array(in_array)
    X = np.arange(-1,2).reshape(-1, 1)

    assert len(in_array)==3
    reg.fit(X,y)
    intercept=reg.intercept_
    slope=reg.coef_

    return intercept/slope


def savgol_filter(df_input,column='confirmed',window=5):
    ''' Savgol Filter which can be used in groupby apply function (data structure kept)

        parameters:
        ----------
        df_input : pandas.series
        column : str
        window : int
            used data points to calculate the filter result

        Returns:
        ----------
        df_result: pd.DataFrame
            the index of the df_input has to be preserved in result
    '''

    degree=1
    df_result=df_input

    filter_in=df_input[column].fillna(0) # attention with the neutral element here

    result=signal.savgol_filter(np.array(filter_in),
                           window, # window size used for filtering
                           1)
    df_result[str(column+'_filtered')]=result
    return df_result

def rolling_reg(df_input,col='confirmed'):
    ''' Rolling Regression to approximate the doubling time'

        Parameters:
        ----------
        df_input: pd.DataFrame
        col: str
            defines the used column
        Returns:
        ----------
        result: pd.DataFrame
    '''
    days_back=3
    result=df_input[col].rolling(
                window=days_back,
                min_periods=days_back).apply(get_doubling_time_via_regression,raw=False)

    return result

def calc_filtered_data(df_input,filter_on='confirmed'):
    '''  Calculate savgol filter and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'

    df_output=df_input.copy() # we need a copy here otherwise the filter_on column will be overwritten

    pd_filtered_result=df_output[['state','country',filter_on]].groupby(['state','country']).apply(savgol_filter)#.reset_index()

    #print('--+++ after group by apply')
    #print(pd_filtered_result[pd_filtered_result['country']=='Germany'].tail())

    #df_output=pd.merge(df_output,pd_filtered_result[['index',str(filter_on+'_filtered')]],on=['index'],how='left')
    df_output=pd.merge(df_output,pd_filtered_result[[str(filter_on+'_filtered')]],left_index=True,right_index=True,how='left')
    #print(df_output[df_output['country']=='Germany'].tail())
    return df_output.copy()

def calc_doubling_rate(df_input,filter_on='confirmed'):
    ''' Calculate approximated doubling rate and return merged data frame

        Parameters:
        ----------
        df_input: pd.DataFrame
        filter_on: str
            defines the used column
        Returns:
        ----------
        df_output: pd.DataFrame
            the result will be joined as a new column on the input data frame
    '''

    must_contain=set(['state','country',filter_on])
    assert must_contain.issubset(set(df_input.columns)), ' Erro in calc_filtered_data not all columns in data frame'


    pd_DR_result= df_input.groupby(['state','country']).apply(rolling_reg,filter_on).reset_index()

    pd_DR_result=pd_DR_result.rename(columns={filter_on:filter_on+'_DR',
                             'level_2':'index'})

    #we do the merge on the index of our big table and on the index column after groupby
    df_output=pd.merge(df_input,pd_DR_result[['index',str(filter_on+'_DR')]],left_index=True,right_on=['index'],how='left')
    df_output=df_output.drop(columns=['index'])


    return df_output


# %%% [2.1] Functions_ quick_plot

def quick_plot(x_in, df_input,y_scale='log',slider=False):
    """ Quick basic plot for quick static evaluation of a time series
    
        you can push selective columns of your data frame by .iloc[:,[0,6,7,8]]
        
        Parameters:
        ----------
        x_in : array 
            array of date time object, or array of numbers
        df_input : pandas dataframe 
            the plotting matrix where each column is plotted
            the name of the column will be used for the legend
        scale: str
            y-axis scale as 'log' or 'linear'
        slider: bool
            True or False for x-axis slider
    
        
        Returns:
        ----------
        
    """
    fig = go.Figure()

    for each in df_input.columns:
        fig.add_trace(go.Scatter(
                        x=x_in,
                        y=df_input[each],
                        name=each,
                        opacity=0.8))
    
    fig.update_layout(autosize=True,
        width=1024,
        height=768,
        font=dict(
            family="PT Sans, monospace",
            size=18,
            color="#7f7f7f"
            )
        )
    fig.update_yaxes(type=y_scale),
    fig.update_xaxes(tickangle=-45,
                 nticks=20,
                 tickfont=dict(size=14,color="#7f7f7f")
                )
    if slider==True:
        fig.update_layout(xaxis_rangeslider_visible=True)
    fig.show(renderer='chrome')
    fig.show()
    


'''
quick_plot(df_analyse.date,
           df_analyse.iloc[:,1:],
           y_scale='linear',
           slider=True)

quick_plot(df_analyse2.date,
           df_analyse2.iloc[:,1:],
           y_scale='linear',
           slider=True)

'''

# %%% [2.2] Function doubling time
def doubling_time(in_array):
    ''' Use a classical doubling time formular, 
     see https://en.wikipedia.org/wiki/Doubling_time '''
    y = np.array(in_array)
    return len(y)*np.log(2)/np.log(y[-1]/y[0])
# %% [4] 



# %% [5] Filter data - Piecewise Linear Regression

country_list=df_analyse2.columns[1:]

## filter data
for each in country_list:
    df_analyse2[each+'_filter']=signal.savgol_filter(df_analyse2[each],
                           5, # window size used for filtering
                           1) # order of fitted polynomial

    
filter_cols= df_analyse2.columns[1:] #['Italy_filter','US_filter', 'Spain_filter', 'Germany_filter']

# %%% [5.1] To check information of the graph

start_pos=50
k = 20
quick_plot(df_analyse2.date[start_pos:],
           df_analyse2[[filter_cols[k],filter_cols[k]+'_filter']].iloc[start_pos:,:],
           #df_analyse2[filter_cols[0]][start_pos:],  
           y_scale='log',
           slider=True)
'''
quick_plot(df_analyse2.date[start_pos:],
           df_analyse2[['','China_filter']].iloc[start_pos:,:], #['US','US_filter']
           y_scale='log',
           slider=True)
'''

# %% [6] Doubling rate data - Piecewise Linear Regression

''' Confirmed data '''
# calculate slope of regression of last x days
# use always a limited number of days to approximate the triangle, attention exponential base assumption
days_back = 3 # this gives a smoothing effect
for pos,country in enumerate(country_list):
    df_analyse2[country+'_DR']=df_analyse2[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)
    
''' Filtered data '''

for pos,country in enumerate(filter_cols):
    df_analyse2[country+'_DR']=df_analyse2[country].rolling(
                                window=days_back,
                                min_periods=days_back).apply(get_doubling_time_via_regression, raw=False)
    
# %%% [5.1] To check information of the graph


 #df_analyse2[['Bhutan','Bhutan_filter','Bhutan_DR','Bhutan_filter_DR']]
 #df_analyse2[[filter_cols[k],filter_cols[k]+'_filter',filter_cols[k]+'_DR',filter_cols[k]+'_filter_DR']].iloc[start_pos:,:]
 
start_pos=50
k = 127
quick_plot(df_analyse2.date[start_pos:],
           #df_analyse2.iloc[start_pos:,[11,12,13,14]]
           df_analyse2[[filter_cols[k],filter_cols[k]+'_filter',filter_cols[k]+'_DR',filter_cols[k]+'_filter_DR']].iloc[start_pos:,:],           
           y_scale='linear',
           slider=True)

test = df_analyse2[[filter_cols[k],filter_cols[k]+'_filter',filter_cols[k]+'_DR',filter_cols[k]+'_filter_DR']].iloc[start_pos:,:]

'''Export data to csv for the Dashboard'''

df_analyse2.to_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_final_set_2.csv',sep=';',index=False)


## 3 Visualization

Visualize data on dashboard 

In [ ]:
# %load /Users/victhorvic/ads_covid-19/src/visualization/visualize_Delivery 3_2.py
#!/usr/bin/env python3
"""
Created on Sat Jul 23 17:00:19 2022

@author: victhorvic
"""


# %% [0] Libraries 
import pandas as pd
import numpy as np

import matplotlib as mpl
import matplotlib.pyplot as plt

import dash
dash.__version__
import dash_core_components as dcc
import dash_html_components as html
from dash.dependencies import Input, Output,State

import plotly.graph_objects as go

import os
print(os.getcwd())

# %% [1] data for Model

#Data from build_features_2.py
df_input_large=pd.read_csv('/Users/victhorvic/ads_covid-19/data/processed/COVID_final_set_2.csv',sep=';')

countries = df_input_large.columns[1:200]

# %% [2] Dash app

'''To refererence links:
    https://medium.com/analytics-vidhya/plotting-multiple-figures-with-live-data-using-dash-and-plotly-4f5277870cd7

    https://plotly.com/python/subplots/
    https://community.plotly.com/t/two-graphs-side-by-side/5312/2
'''
fig = go.Figure()

app = dash.Dash()
app.layout = html.Div([

    dcc.Markdown('''
    #  Applied Data Science on COVID-19 data

    Goal of the project is to teach data science by applying a cross industry standard process,
    it covers the full walkthrough of: automated data gathering, data transformations,
    filtering and machine learning to approximating the doubling time, and
    (static) deployment of responsive dashboard.

    '''),

    dcc.Markdown('''
    ## Multi-Select Country for visualization
    '''),


    dcc.Dropdown(
        id='country_drop_down',
        options=[ {'label': each,'value':each} for each in countries],
        value=['US', 'Germany','Mexico'], # which are pre-selected
        multi=True
    ),

    dcc.Markdown('''
        ## Select Timeline of confirmed COVID-19 cases or the approximated doubling time
        '''),


    dcc.Dropdown(
    id='doubling_time',
    options=[
        {'label': 'Timeline Confirmed ', 'value': 1},
        {'label': 'Timeline Confirmed Filtered', 'value': 2},
        {'label': 'Timeline Doubling Rate', 'value': 3},
        {'label': 'Timeline Doubling Rate Filtered', 'value': 4},
    ],
    value=1,
    multi=False
    ),

    dcc.Graph(figure=fig, id='main_window_slope')
])

# %% [3] Callback function 

@app.callback(
    Output('main_window_slope', 'figure'),
    [Input('country_drop_down', 'value'),
    Input('doubling_time', 'value')])
def update_figure(country_list,show_doubling):


    if show_doubling > 2:
        my_yaxis={'type':"log",
               'title':'Approximated doubling rate over 3 days (larger numbers are better #stayathome)'
              }
    else:
        my_yaxis={'type':"log",
                  'title':'Confirmed infected people (source johns hopkins csse, log-scale)'
              }


    traces = []
    for each in country_list:
        
        if show_doubling == 2: 
            df_plot=df_input_large[each+'_filter']
        elif show_doubling == 3: 
            df_plot=df_input_large[each+'_DR']
        elif show_doubling == 4: 
            df_plot=df_input_large[each+'_filter_DR']
        else:
            df_plot=df_input_large[each]

        traces.append(dict(x=df_input_large.date,
                                y=df_plot,
                                mode='markers+lines',
                                opacity=0.9,
                                name=each
                        )
                )

    return {
            'data': traces,
            'layout': dict (
                width=1280,
                height=720,

                xaxis={'title':'Timeline',
                        'tickangle':-45,
                        'nticks':20,
                        'tickfont':dict(size=14,color="#7f7f7f"),
                      },

                yaxis=my_yaxis
        )
    }

# %% [4] Run dashboard
app.run_server(debug=True, use_reloader=False)

